# Setting

In [8]:
import sys, os
import pandas as pd

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as iC
from Clust.clust.ingestion.mongo import mongo_client
from Clust.clust.ML.common import ML_api

db_client = iC.InfluxClient(ins.CLUSTDataServer2)
mongodb_client = mongo_client.MongoClient(ins.CLUSTMetaInfo2)

In [9]:
# feature list와 data_y_flag는 model_info에서 받아와야 합니다.
# regression
param1 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_testX',
        "feature_list":['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7',
       'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7',
       'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_testy',
        "feature_list":["value"]
    },
    'data_y_flag' : 'true',
    'model_name':'regression_energy_cleanLevel4_trainX_regression_LSTM_rg_'
}

# forecasting
param2 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_testX',
        "feature_list":['Temperature', 'out_temp','sin_hour']
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_testX',
        "feature_list":['Temperature']
    },
    'data_y_flag' : 'false',
    'model_name':'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX_regression_LSTM_rg_'
}

# classification
param3 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'classification_actionPattern_cleanLevel0_testX',
        "feature_list":['col_0', 'col_1','col_2','col_3','col_4','col_5','col_6','col_7','col_8']
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'classification_actionPattern_cleanLevel0_testy',
        "feature_list":['value']
    },
    'data_y_flag' : 'true',
    'model_name':'classification_actionPattern_cleanLevel0_trainX_classification_LSTM_cf_'
}

In [10]:
params = param1

# chage tpye string to bool -> ex) 'true' -> True
params = ML_api.chagne_type_str_to_bool(params)

In [11]:
# get model meta
model_name = params['model_name']
meta_file_name = "meta.json"
from Clust.clust.ML.tool import meta
#model_meta = meta.read_model_meta_from_mongodb(mongodb_client, 'model', 'meta', model_name)
model_meta = meta.read_model_meta_from_local(meta_file_name, model_name)

In [12]:
model_meta

{'ingestion_param_X': {'bucket_name': 'integration',
  'ms_name': 'regression_energy_cleanLevel4_trainX',
  'feature_list': ['Press_mm_hg',
   'RH_1',
   'RH_2',
   'RH_3',
   'RH_4',
   'RH_5',
   'RH_6',
   'RH_7',
   'RH_8',
   'RH_9',
   'RH_out',
   'T1',
   'T2',
   'T3',
   'T4',
   'T5',
   'T6',
   'T7',
   'T8',
   'T9',
   'T_out',
   'Tdewpoint',
   'Visibility',
   'Windspeed']},
 'ingestion_param_y': {'bucket_name': 'integration',
  'ms_name': 'regression_energy_cleanLevel4_trainy',
  'feature_list': ['value']},
 'data_y_flag': True,
 'scaler_param': {'scaler_flag': 'scale',
  'scale_method': 'minmax',
  'scaler_path': './scaler/',
  'scaler_file_path': {'XScalerFile': {'fileName': 'scaler.pkl',
    'filePath': './scaler/regression_energy_cleanLevel4_trainX/minmax/900878d7bba6f2a4017d1c3399909ea7/scaler.pkl'},
   'yScalerFile': {'fileName': 'scaler.pkl',
    'filePath': './scaler/regression_energy_cleanLevel4_trainy/minmax/f69156750a210491ffd4a67b605bc88b/scaler.pkl'}}},


In [6]:
# make test data, scaler
test_X_array, test_y_array, scaler_X, scaler_y = ML_api.test_data_preparation(params, model_meta, db_client)

{'bucket_name': 'integration', 'ms_name': 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_testX', 'feature_list': ['Temperature', 'out_temp', 'sin_hour']}
nan_limit_num:  12
Original num: 1003 Final num: 977 NaN num: 26


In [7]:
# set scaler, feature list by data y flag
scaler_feature_dict = {}
if model_meta['data_y_flag']:
    scaler_feature_dict['scaler'] = scaler_y
    scaler_feature_dict['feature_list'] = model_meta['ingestion_param_y']['feature_list']
    scaler_feature_dict['target'] = model_meta['ingestion_param_y']['feature_list'][0]

else:
    scaler_feature_dict['scaler'] = scaler_X
    scaler_feature_dict['feature_list'] = model_meta['ingestion_param_X']['feature_list']
    scaler_feature_dict['target'] = model_meta['ingestion_param_X']['feature_list'][0]

In [8]:
# model test
result = ML_api.ML_test(model_meta, test_X_array, test_y_array, scaler_feature_dict)

features shape: torch.Size([977, 24, 3]) targets shape:  torch.Size([977, 1])

Start testing data

scale


In [9]:
result

{'df_result':          value  prediction
 0    27.580000   26.567505
 1    27.820001   26.691312
 2    27.980001   26.827189
 3    28.100000   26.984124
 4    28.220000   27.146790
 ..         ...         ...
 971  20.960000   20.405946
 972  21.499999   20.470601
 973  21.859999   20.561689
 974  22.100000   20.742891
 975  22.340000   20.997500
 
 [976 rows x 2 columns],
 'result_metrics': {'mae': 0.5075732639944943,
  'rmse': 0.6328042145868005,
  'r2': 0.9032548237499973,
  'mape': 2.0442301939238505}}

In [10]:
model_meta['model_info']

{'model_purpose': 'regression',
 'model_method': 'LSTM_rg',
 'model_name': 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX_regression_LSTM_rg_',
 'model_tags': 'tagstest',
 'train_parameter': {'lr': 0.0001,
  'weight_decay': 1e-06,
  'n_epochs': 5,
  'batch_size': 16,
  'device': 'cpu'},
 'model_parameter': {'rnn_type': 'lstm',
  'input_size': 3,
  'hidden_size': 64,
  'num_layers': 2,
  'output_dim': 1,
  'dropout': 0.1,
  'bidirectional': True},
 'seq_len': 24,
 'input_size': 3,
 'model_file_path': {'modelFile': {'fileName': 'model.pth',
   'filePath': './Models/LSTM_rg/forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX_regression_LSTM_rg_/forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX/model.pkl'}}}